In [1]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import hashlib
import pandas as pd
import time as t
import hashlib
import json

C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
weekdays = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
home = 'http://www.stromboli.at'
events = '/index.php/programm'

pages = []
pages.append(home + events)

response = requests.get(home + events)
soup = BeautifulSoup(response.content, 'html.parser')
events = []

for tt in range(len(soup.select('.pagination')[0]('a'))):
    try:
        int(soup.select('.pagination')[0]('a')[tt].attrs['title'])
        pages.append(home + soup.select('.pagination')[0]('a')[tt].attrs['href'])
    except: KeyError

for page in pages:
    t.sleep(3)
    response = requests.get(page)
    soup = BeautifulSoup(response.content, 'html.parser')

    i = 0

    items = soup.select('h2.article-title')
    for item in items:
        t.sleep(0.1)

        try:
            title_list = [item.select('a')[0].get_text().strip().split()[2][5:], ' '.join(item.select('a')[0].get_text().strip().split()[3:])]
            title = ' '.join(title_list)
            if title[0] == '-':
                title = title[6:]
            if title == ' ':
                title = soup.select('div.col-md-8')[2*i]('p')[0].get_text().strip().split('\r')[0]
                
        except: AttributeError

        try:
            date = item.select('a')[0].get_text().split()[1]
            day, month, year = int(date[0:2]), int(date[3:5]), int('20'+date[6:8])
            date = datetime.datetime(year, month, day)
            week_day = weekdays[date.weekday()]
        except: AttributeError 

        try:    
            time = item.select('a')[0].get_text().split()[2][0:5]
        except: AttributeError 

        try:
            link = home + item.select('a')[0].attrs['href']
        except: AttributeError 

        try:
            location = 'Stromboli Hall in Tirol'
        except: AttributeError

        try:
            short = soup.select('div.col-md-8')[2*i]('p')[0].get_text().strip()
        except: AttributeError

        i += 1


        hash = hashlib.sha256()
        hash.update(bytes(title, 'utf-8'))
        hash.update(bytes(location, 'utf-8'))
        hash.update(bytes(str(date), 'utf-8'))
        hash.update(bytes(str(time), 'utf-8'))
        
        if type(date) == datetime.datetime:
            
            events.append({
                "name": title,
                "date": date,
                "location": location,
                "link": link,
                "short": short,
                "source": "Stromboli Hall in Tirol Homepage",
                "identifier": hash.hexdigest()
            })

In [4]:
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine

Base = declarative_base()

class Event(Base):
    __tablename__ = 'event'

    id = Column(Integer, primary_key=True)
    identifier = Column(String)
    name = Column(String)
    location = Column(String)
    source = Column(String)
    short = Column(String)
    date = Column(DateTime)
    link = Column(String)



engine = create_engine('sqlite:///events_db_ue6_scraper_ownly.sqlite')
Base.metadata.create_all(engine)

In [5]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
# from declarations import Event, Base
# from fhku_event_scraper import get_latest_events

engine = create_engine('sqlite:///events_db_ue6_scraper_ownly.sqlite')
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
session = DBSession()


for event in events:
    ev = session.query(Event).filter_by(identifier=event['identifier']).first()
    
    if not ev:
        session.add(Event(
            name=event['name'],
            location=event['location'],
            link=event['link'],
            short=event['short'],
            date=event['date'],
            source=event['source'],
            identifier=event['identifier']
            ))

        

session.commit()
session.close()